## Calculate Phi spatial average within a basin or within LMEs

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
dset = xr.open_dataset(filename)
dz = dset['dz']/1e2 # m
darea = dset['TAREA'].isel(nlat=range(a1,a2),nlon=range(b1,b2))/1e10 #km*km
KMT = dset['KMT'].isel(nlat=range(a1,a2),nlon=range(b1,b2))
darea = darea*dz/dz

In [4]:
darea = darea.transpose("z_t","nlat","nlon")

In [5]:
for xx in range(a2-a1):
    for yy in range(b2-b1):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            darea[kk:, xx, yy] = float("nan")

In [6]:
#varr = 'O2'
varr = 'AOU'
#varr = 'TEMP'
#varr = 'SALT'

In [7]:
filename = f'{path}/DPLE_driftcorrected_Phi_components_{varr}_ens_mean_monthly_new.nc'
ds = xr.open_dataset(filename)

In [8]:
ds

<xarray.Dataset>
Dimensions:    (z_t: 35, nlat: 140, nlon: 195, Y: 64, L: 122)
Coordinates:
  * z_t        (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT       (nlat, nlon) float64 ...
    ULONG      (nlat, nlon) float64 ...
    ULAT       (nlat, nlon) float64 ...
    TLONG      (nlat, nlon) float64 ...
  * Y          (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * L          (L) int64 1 2 3 4 5 6 7 8 9 ... 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_AOU_0  (L, z_t, nlat, nlon, Y) float64 ...
    Phi_AOU_1  (L, z_t, nlat, nlon, Y) float64 ...
    Phi_AOU_2  (L, z_t, nlat, nlon, Y) float64 ...

In [9]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask']
lme = [1,2,3,4,5,6,7,8,9,10,65]

In [10]:
nyear = ds.dims['Y']
nlead = ds.dims['L']
nver = 35
dsf = xr.Dataset()
#for ii in range(9):
#for ii in range(13):
for ii in range(3):
    dsf[f'Phi_{varr}_{ii}'] = ds[f'Phi_{varr}_{ii}']*darea[:35,:,:]

In [11]:
for ii in range(3):
    dsf[f'Phi_{varr}_{ii}'] = ds[f'Phi_{varr}_{ii}'].transpose("Y","L","z_t","nlat","nlon")

In [12]:
#phi_box = np.ma.zeros([9, len(lme), nmon, nver])
#phi_box = np.ma.zeros([13, len(lme), nyear, nlead, nver])
phi_box = np.ma.zeros([3, len(lme), nyear, nlead, nver])
phi_box.mask = True
# select a LME
for box,nn in zip(lme,range(len(lme))):
    print(box)
    latloc, lonloc = np.where(lme_mask.isel(nlat=range(a1,a2),nlon=range(b1,b2)) == box)
    ar1 = np.ma.zeros([len(latloc),nver]);ar1.mask=True
    #ar2 = np.ma.zeros([13, len(latloc),nyear,nlead,nver]);ar2.mask=True
    ar2 = np.ma.zeros([3, len(latloc),nyear,nlead,nver]);ar2.mask=True
    for bb in range(len(latloc)):
        ar1[bb] = darea.isel(z_t=range(35),nlat=latloc[bb], nlon=lonloc[bb])
        #for ii in range(13):
        for ii in range(3):
            ar2[ii,bb] = dsf[f'Phi_{varr}_{ii}'].isel(nlat=latloc[bb], nlon=lonloc[bb])
    #
    tarea = np.nansum(ar1,axis=0)
    #tarea = np.tile(tarea[np.newaxis,np.newaxis,np.newaxis,:],[13,nyear,nlead,1])
    tarea = np.tile(tarea[np.newaxis,np.newaxis,np.newaxis,:],[3,nyear,nlead,1])
    phi_box[:,nn] = np.nansum(ar2,axis=1)/tarea   

1
2
3
4
5
6
7
8
9
10
65


In [13]:
dsf = xr.Dataset()
#for ii in range(9):
#for ii in range(13):
for ii in range(3):
    dsf[f'Phi_{varr}_{ii}'] = xr.DataArray(data=phi_box[ii], coords=dict(lme=('lme', lme), Y=('Y', ds.coords['Y'].values), L=('L', ds.coords['L'].values), z_t=('z_t', ds.coords['z_t'].values)), dims=('lme','Y','L','z_t'))

In [14]:
dsf.load()

<xarray.Dataset>
Dimensions:    (lme: 11, Y: 64, L: 122, z_t: 35)
Coordinates:
  * lme        (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y          (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * L          (L) int64 1 2 3 4 5 6 7 8 9 ... 115 116 117 118 119 120 121 122
  * z_t        (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
Data variables:
    Phi_AOU_0  (lme, Y, L, z_t) float64 -1.201e-06 -1.197e-06 ... -4.759e-07
    Phi_AOU_1  (lme, Y, L, z_t) float64 -2.956e-06 -2.946e-06 ... -1.174e-06
    Phi_AOU_2  (lme, Y, L, z_t) float64 -7.328e-06 -7.303e-06 ... -2.895e-06

In [15]:
dsf.to_netcdf(f'{path}/DPLE_driftcorrected_Phi_components_{varr}_Eo_space_horizontal_mean_LME11_ens_mean_monthly.nc', mode='w')